<h1 style="text-align: center; font-size:2.5em;"> Assignement 2 </h1>

# <u> Part 1: Twitter Network Analysis</u>

### <i> Exercise 1: Build the network of retweets.</i>

<hr>

### <i> Exercise 2: Visualize the network of retweets and investigate differences between the parties. </i>

<hr>

### <i> Exercise 3: Community detection.</i>

<hr>

# <u> Part 2: What do republican and democratic members tweet about? </u>

### <i> Exercise 4: TF-IDF of the republican and democratic tweets. </i>

<hr>

# <u> Part 3: Sentiment analysis</u>

### <i> 
Exercise 5: Sentiment over the Twitter data.